## <center>MSM-building Calculations</center>

##### MUST SHARE KERNEL WITH NOTEBOOK `00-load-data.ipynb`

This notebook loads tons of stuff into memory. There is some code in place elsewhere to just load and toss large data objects for each operation but this takes forever compared to storing data in memory and plotting from it using a computer with suitable RAM.

In [22]:
# using Timer context managers
# create one then call to late-bind
# a description of the usage
time_loadfile = "loadfile"
time_calculation = "calculate"

kwargs_tica = ["lag"]
kwargs_kmeans = ["k"]

da = analysis_directory

if not da.is_dir():
    
    da.mkdir(parents=True)
    
blueback = colorama.Back.BLUE
yellowback = colorama.Back.LIGHTYELLOW_EX
whitetxt = colorama.Fore.WHITE
bluetxt = colorama.Fore.BLUE
blacktxt = colorama.Fore.BLACK
whiteback = colorama.Back.WHITE
resetback = colorama.Back.RESET
resettxt = colorama.Fore.RESET
rreset = resetback + resettxt

ts1 = whiteback+bluetxt
is1 = blueback+whitetxt


def _thisone_tica(feat, setup):

    return _model_name(feat, {
        **{k:setup["kwargs"][k] for k in kwargs_tica},
        **({"bes":f"{(setup['epochsize']*ns_per_step):.0f}"}
           if "epochsize" in setup else {}),
    })


def _thisone_kmeans(feat, setup):
    
    return _model_name(feat, {
        **{f"tica_{k}": setup["input"]["kwargs"][k] for k in kwargs_tica},
        **{k          : setup["kwargs"][k]          for k in kwargs_kmeans},
    })


def _thisone_mits(feat, setup):

    inpsetup = setup["input"]
    
    return _model_name(feat, {
        **{f"tica_{k}": inpsetup["input"]["kwargs"][k] for k in kwargs_tica},
        **{f"kmeans_{k}": inpsetup["kwargs"][k] for k in kwargs_kmeans},
        **{"lags": _clean_list(setup["kwargs"]["lags"])},
    })


def _thisone_msm(feat, setup):
    
    inpsetup = setup["input"]
    
    return _model_name(feat, {
        **{f"tica_{k}": inpsetup["input"]["kwargs"][k] for k in kwargs_tica},
        **{f"kmeans_{k}": inpsetup["kwargs"][k] for k in kwargs_kmeans},
        **{"lag": setup["kwargs"]["lag"]},
    })


def _thisone_pcca(feat, setup):
    
    inpsetup = setup["input"]
    inpsetupinp = inpsetup["input"]
    
    return _model_name(feat, {
        **{f"tica_{k}": inpsetupinp["input"]["kwargs"][k] for k in kwargs_tica},
        **{f"kmeans_{k}": inpsetupinp["kwargs"][k] for k in kwargs_kmeans},
        **{"lag": inpsetup["kwargs"]["lag"],
           "n_macrostate": setup["kwargs"]["m"]},
    })

#### TICA and VAMP

------------------------

In [ ]:
for n, (feat, name, dataset) in enumerate(aswa_tools.iter_models(all_models)):
    
    dds = da / name
    
    dtica = dds / "tica"
    ftica = dtica / "tica.pyemma"

    if not dds.is_dir():
        dds.mkdir()
    if not dtica.is_dir():
        dtica.mkdir()
    
    for m, setup in enumerate(dataset["tica"]):

        timer = setup["timer"] = Timer(#Heat.start_one(
            description=True, verbose=True)

        setup["hash"] = thisone = _thisone_tica(feat, setup)
        _ftica = ftica.with_suffix(".%s" % thisone)

        if _ftica.is_file():
            print(f"Loading File: {_ftica}")
            with timer(yellowback+blacktxt+time_loadfile+rreset):
                setup["result"] = pyemma.load(_ftica)

        else:
            print(f"Calculating and Saving to File: {_ftica}")
            try:
                with timer(time_calculation):
                    setup["result"] = coor.tica(
                        setup["input"], **setup["kwargs"])
                    
                setup["result"].save(_ftica, save_streaming_chain=True)

            except:
                print(yellowback+blacktxt+"Calculation failed!"+rreset)
                print(colorama.Back.LIGHTRED_EX+blacktxt+traceback.format_exc()+rreset)

                
print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")

#### k-means and regular-space clusterings
------------------------

In [ ]:
for n, (feat, nm, dataset) in enumerate(aswa_tools.iter_models(all_models)):
    
    dds = da / nm

    for setup in dataset["cluster_kmeans"]:

        dkmeans = dds / "kmeans"
        fkmeans = dkmeans / "kmeans.pyemma"

        if not dkmeans.is_dir():
            dkmeans.mkdir(parents=True)

        inp = setup["input"]["result"]
        setup["hash"] = thisone = _thisone_kmeans(feat, setup)
        _fkmeans = fkmeans.with_suffix(".%s" % thisone)
        
        timer = setup["timer"] = Timer(
            description=True, verbose=True)

        if _fkmeans.is_file():
            print(f"Loading File: {_fkmeans}")
            with timer(yellowback+blacktxt+time_loadfile+rreset):
                setup["result"] = pyemma.load(_fkmeans)

        else:
            if not inp: continue

            print(f"Calculating and Saving to File: {_fkmeans}")
            inp._default_chunksize = 10000
            tica_inp = inp.get_output(list(range(n_tica_dim)))
            with timer(yellowback+blacktxt+time_calculation+rreset):
                setup["result"] = coor.cluster_kmeans(
                    tica_inp, **setup["kwargs"])

            setup["result"].save(_fkmeans, save_streaming_chain=True)


print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")

#### Many MSMs calculated to examine Implied TimeScales
----------------------------------

In [ ]:
from pyemma.msm.estimators import ImpliedTimescalesMatrix

stepname = "msm"
thiscalc = "mits"
disc_methods = ["kmeans"]

for feat, name, dataset in aswa_tools.iter_models(all_models):
    
    dds   = da / name
    pmits = dds / stepname
    fmits = pmits / f"{thiscalc}.pyemma"
    dmits = dataset[thiscalc] = list()

    if not pmits.is_dir():
        pmits.mkdir(parents=True)

    print(ts1+f"MSMs for workflow: {name}"+rreset)
    for disc_method in disc_methods:
        for inpsetup in dataset[f"cluster_{disc_method}"]:
          
            curmits = dict()
            curmits["input"] = inpsetup
            curmits["timer"] = timer = Timer(description=True, verbose=True)
            curmits["kwargs"] = dict(
                lags=msm_lags,
                reversible=reversible,
                nits=10,
                errors="bayes",
                nsamples=20,
               # dt_traj=f"{dataset['timestep']:0.2} ns",
                weights="empirical",  # if nm=="ltj" else "oom",
            )
            
            resmits = None
            curmits["hash"] = thisone = _thisone_mits(feat, curmits)
            _fmits = fmits.with_suffix(f".{thisone}")
            
            if not _fmits.is_file():
                
                if not inpsetup["result"]: continue
                print(f"{is1}Calculating & Saving:{rreset} {thisone}")
                with timer(yellowback+blacktxt+time_calculation+rreset):
                    resmits = pyemma.msm.its(inpsetup["result"].dtrajs, **kwargs)

                resmits.save(_fmits)

            else:
                
                print(is1+f"Loading File:{rreset} {thisone}")
                with timer(yellowback+blacktxt+time_loadfile+rreset):
                    resmits = pyemma.load(_fmits)

            resmatrix = ImpliedTimescalesMatrix(resmits, n_macrostates, msm_lags)
            curmits["result"] = resmatrix
            
            dmits.append(curmits)
            
            
print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")

### Index the msms separate from ITS objects for massive convenience
  - just making separate object references to cleanup access
----------------

In [ ]:
for feat, name, dataset in aswa_tools.iter_models(all_models):
    
    mms = dataset["msm"] = list()
    
    for setup in dataset["mits"]:
        for mm in setup["result"].its.models:

            msmsetup = dict()
            msmsetup["timer"] = None
            msmsetup["input"] = setup["input"]
            msmsetup["result"] = mm
            msmsetup["kwargs"] = {k:v for k,v in setup["kwargs"].items() if k != "lags"}
            msmsetup["kwargs"].update({"lag": mm.lag})
            msmsetup["hash"] = thisone = _thisone_msm(feat, msmsetup)
            mms.append(msmsetup)
            

print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")

### PCCA Calculation and Indexing
-----------------

In [ ]:
for feat, name, dataset in aswa_tools.iter_models(all_models):
    
    dds   = da / name
    ppcca = dds / "pcca"
    fpcca = ppcca / "pcca.pyemma"
    dpcca = dataset["pcca"] = list()

    if not ppcca.is_dir():
        ppcca.mkdir(parents=True)

    print(ts1+f"MSMs for workflow: {name}"+rreset)
    for nma in n_macrostates:
        for msmsetup in dataset["msm"]:

            pccasetup = dict()
            pccasetup["input"] = inp = msmsetup
            pccasetup["kwargs"] = dict(m=nma)
            pccasetup["timer"] = timer = Timer(description=True, verbose=True)
            pccasetup["hash"] = thisone = _thisone_pcca(feat, pccasetup)
            
            _fpcca = fpcca.with_suffix(f".{thisone}")
            respcca = None
            
            if _fpcca.is_file():
                if not inp["result"]: continue
                print(is1+f"Loading File: {thisone}"+rreset)
                with timer(yellowback+blacktxt+time_loadfile+rreset):
                    respcca = pyemma.load(_fpcca)
                
            else:
                print(is1+f"Calculating & Saving: {thisone}"+rreset)
                try:
                    with timer(yellowback+blacktxt+time_calculation+rreset):
                        respcca = inp["result"].pcca(nma)

                    respcca.save(_fpcca)
                    
                except:
                    print(yellowback+blacktxt+"Calculation failed!"+rreset)
                    print(colorama.Fore.LIGHTRED_EX+blacktxt+traceback.format_exc()+rreset)
                
            pccasetup["result"] = respcca
            dpcca.append(pccasetup)
            
            
print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")

### HMM Calculation
-----------------

### Reorganize analysis profiling data

------------------

In [21]:
times = dict()

for feat, name, dataset in aswa_tools.iter_models(all_models):
    times[name] = dict()
    for calc in ("tica", "cluster_kmeans", "msm", "pcca", "hmm"):

        times[calc] = dict()
        
        if isinstance(dataset[calc], dict):
            iterrer = dataset[calc].items()
        elif isinstance(dataset[calc], list):
            if calc == "tica": itto = _thisone_tica
            elif calc == "cluster_kmeans": itto = _thisone_kmeans
            else: raise ValueError
            iterrer = iter((itto(feat, setup), setup) for setup in dataset[calc])
        else:
            raise TypeError
            
        for thisone, setup in iterrer:
            pass

ValueError: 

------------------

------------------

------------------

------------------

------------------

------------------

------------------

In [ ]:
'''
for n, (feat, name, dataset) in enumerate(
    aswa_tools.iter_models(all_models)
):
    for msmsetup in dataset["msm"].items():
        
    dds = da / nm
    dmits = dataset.setdefault("mits", dict())

    for disc_method in ["kmeans"]:
        
        pmits = dds / "msm"
        fmits = pmits / "mits.pyemma"

        if not pmits.is_dir():
            pmits.mkdir(parents=True)

        for i, inpsetup in enumerate(
            dataset["cluster_%s" % disc_method]
        ):
            chosen_tica_lag = inpsetup["input"]["kwargs"]["lag"]
            chosen_k = inpsetup["kwargs"]["k"]
            inp = inpsetup["result"]

            thisone = _model_naame(feat, {
                "tica_lag":     chosen_tica_lag,
                "k":            chosen_k,
                "tica_weights": "empirical" if nm=="ltj" else "koopman",
                "reversible":   reversible,
                "disc":         disc_method,
                "msm_lags":     _clean_list(msm_lags),
            })
'''

In [ ]:
'''     for m, setup in enumerate(dataset["cluster_regspace"]):
            thisone = _model_name(feat, setup["kwargs"])

            # clear_output(wait=True)
            print(colorama.Back.CYAN + "Next is regspace clustering TICA")
            print(
                colorama.Back.CYAN
                + "Calculated %d models so far" % (n_kmeans * (1 + n) + m + n_regspaces * n)
            )
            print((" - on to model %s" % thisone) + colorama.Back.RESET)
            print(
                colorama.Fore.LIGHTGREEN_EX
                + " - number %d / %d (wrong 2nd number)"
                % (n_kmeans * n + m, (n_kmeans + n_regspaces) * len(all_models))
            )

            _fregspace = fregspace.with_suffix(".%s" % thisone)

            setup["input"] = aswa_tools.get_matching_input(
                dataset["tica"], key="lag", val=chosen_tica_lag
            )

            if not setup["input"]:
                continue

            # SUM TING WONG with TICA object
            setup["input"]._default_chunksize = 10000

            tica_inp = setup["input"].get_output(list(range(n_tica_dim)))

            mindist = (
                np.power(
                    np.product(
                        [
                            mx - mn
                            for mn, mx in zip(
                                np.min(np.concatenate(tica_inp), axis=0),
                                np.max(np.concatenate(tica_inp), axis=0),
                            )
                        ]
                    ),
                    1.0 / n_tica_dim,
                )
                / 25.0
            )

            setup["actual-dmin"] = setup["kwargs"]["dmin"] * mindist

            if _fregspace.is_file():
                print("Found Results File: {}".format(_fregspace))

                setup["result"] = pyemma.load(_fregspace)

            else:

                print("Calculating and Saving to File: {}".format(_fregspace))

                setup["result"] = aswa_tools.refined_regspace(
                    tica_inp, mindist, **setup["kwargs"]
                )

                setup["result"].save(_fregspace, save_streaming_chain=True)
'''


In [ ]:
"""
for n, (feat, nm, dataset) in enumerate(aswa_tools.iter_models(all_models)):

    dmits = dataset.setdefault("mits", dict())
    dpits = dataset.setdefault("pits", dict())
    dhits = dataset.setdefault("hits", dict())

    dds = da / nm

    for chosen_tica_lag in chosen_tica_lags:
        for m, (clust_method, par_key) in enumerate(
            [("regspace", "dmin"), ("kmeans", "k")]
        ):
            mits = dmits.setdefault(clust_method, list())
            pits = dpits.setdefault(clust_method, list())
            hits = dhits.setdefault(clust_method, list())

            # MSM
            pmits = dds / "msm"
            fmits = pmits / "mits.pyemma"

            # PCCA
            ppits = dds / "pcca"
            fpits = ppits / "pits.pyemma"

            # HMM
            phits = dds / "hmm"
            fhits = phits / "hits.pyemma"

            for d in {pmits, ppits, phits}:
                if not d.is_dir():
                    d.mkdir(parents=True)

            for i, setup in enumerate(dataset["cluster_%s" % clust_method]):

                _fmits = fmits.with_suffix(
                    ".%s__msmlags-%s"%(
                        _model_name(feat,
                            reduce(lambda x, y: dict(x, **y), (
                                {"weights": "empirical"
                                 if nm == "ltj" else "koopman"},
                                {"reversible": reversible},
                                {"disc": clust_method},
                                setup["kwargs"]))),
                        _clean_list(msm_lags))
                )

                # MSM MITS
                #   FIXME --> these don't check if exist already
                curmits = dict()
                curmits["par"] = (par_key, setup["kwargs"][par_key])
                curmits["kwargs"] = dict(
                    lags=msm_lags,
                    reversible=reversible,
                    nits=10,
                    errors="bayes",
                    weights="empirical",  # if dataset["n_trajs"]==1 else "oom",
                )

                if not _fmits.is_file():
                    if not setup["result"]:
                        continue

                    with timer(timer_calculate):
                        resmits = pyemma.msm.its(
                            setup["result"].dtrajs,
                            nsamples=20,
                            **curmits["kwargs"])

                    resmits.save(_fmits)

                else:
                    resmits = pyemma.load(_fmits)

                curmits["result"] = resmits

                mits.append(curmits)

                # PCCA PITS
                for msm in resmits.models:
                    for nma in n_macrostates:
                        _fpits = fpits.with_suffix("%s__%s"%(
                            _model_name(
                                "msmlag": msm.lag,
                                "nstates": nma,
                            ), thisone,)
                        )
            
            # HMM HITS
            
            
            
            
            
            
            
            # PCCA COMPLICATED HERE
            for msm in result.models:
                for nma in n_macrostates:
                    thisone = "%s__nstates-%r"%("%s-%d-%s" % ("msmlag-",
                        msm.lag, _model_name(feat, reduce(
                            lambda x, y: dict(x, **y), (
                                {"weights": "empirical" if nm is "ltj" else "koopman"},
                                {"reversible": reversible},
                                {k:v for k,v in setup["kwargs"].items() if k != "lags"})))), nma)
            
                    _fpcca = fpits.with_suffix(".%s-%s-%d__%s" % (
                        clust_method, par_key, setup["par"][1], thisone))

                    if _fpcca.is_file():
                        resultlist.append(pyemma.load(_fpcca))
                    else:
                        result = msm.pcca(nma)
                        result.save(_fpcca)
                        resultlist.append(result)
                
                
                
                
            # TODO PCCA HMM

print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")
"""

In [ ]:
"""for n, (feat, nm, dataset) in enumerate(aswa_tools.iter_models(all_models)):

    dlits = dataset.get("its", dict())
    dpits = dataset.get("pits", dict())
    dhits = dataset.get("hits", dict())

    for m, (clust_method, par_key) in enumerate(
        [("regspace", "dmin"), ("kmeans", "k")]
    ):
        lits = dlits.get(clust_method, list())
        pits = dpits.get(clust_method, list())
        hits = dhits.get(clust_method, list())

        # MSM
        dds = da / nm
        dmsm = dds / "msm" / ("tica_lag_%d" % chosen_tica_lag)
        fits = dmsm / "its-results.pyemma"

        # PCCA
        dpits = dds / "pcca" / ("tica_lag_%d" % chosen_tica_lag)
        fpits = dpits / "pits-results.pyemma"
        
        # HMM
        dhits = dds / "hmm" / ("tica_lag_%d" % chosen_tica_lag)
        fhits= dhits / "hits-results.pyemma"
        
        for d in {dmsm, dpits, dhmm}:
            if not d.is_dir():
                d.mkdir(parents=True)

                
        for i, setup in enumerate(dataset["cluster_%s" % clust_method]):

            thisone = "%s-%s-%s" % (
                "msmlags",
                ",".join([str(d) for d in msm_lags]),
                _model_name(
                    feat,
                    reduce(
                        lambda x, y: dict(x, **y),
                        (
                            {"weights": "empirical" if nm is "ltj" else "koopman"},
                            {"reversible": reversible},
                            setup["kwargs"],
                        ),
                    ),
                ),
            )

            # clear_output(wait=True)
            print(
                colorama.Back.CYAN
                + "Now creating MSMs using clusterings"
                + "as the discretization to state space"
            )
            print(colorama.Back.CYAN + "Calculated %d models so far" % (m))
            print((" - on to model %s" % thisone) + colorama.Back.RESET)
            print(
                colorama.Fore.LIGHTGREEN_EX
                + " - number %d / %d (wrong 2nd number)"
                % (n_kmeans * n + m, (n_kmeans + n_regspaces) * len(all_models))
            )

            _fits = fits.with_suffix(".%s-%s" % (clust_method, thisone))
            dits = dict()
            dits["par"] = (par_key, setup["kwargs"][par_key])
            dits["kwargs"] = dict(
                lags=msm_lags,
                reversible=reversible,
                nits=10,
                errors="bayes",
                weights="empirical",  # if dataset["n_trajs"]==1 else "oom",
            )

            if not _fits.is_file():
                
                if not setup["result"]:
                    continue

                print(colorama.Back.BLUE + colorama.Fore.BLACK + "drajs shape:")
                print(str(len(setup["result"].dtrajs)))
                print(
                    pformat([dt.shape for dt in setup["result"].dtrajs])
                    + colorama.Back.RESET
                )
                print(
                    colorama.Fore.LIGHTGREEN_EX
                    + "Calculating and Saving to File: {}".format(_fits)
                )

                result1 = pyemma.msm.its(
                    # dits["result"] = pyemma.msm.its(
                    # dits["input"],
                    setup["result"].dtrajs,
                    nsamples=20,
                    **dits["kwargs"])
                
                # could set up as a filepath that gets
                # loaded as needed --> DMGR!!!
                #dits["result"] = _fits
                result.save(_fits)

            else:
                print("Found Results File: {}".format(_fits))
                result = pyemma.load(_fits)
                # dits["result"] = _fits

   #         # Reverse so both with increasing N clusters
    #        if "regspace" in clust_method:
     #           lits.insert(0, dits)
      #      else:
       #         lits.append(dits)
                
            dits["result"] = result
            
            # PCCA COMPLICATED HERE
            for msm in result.models:
                for nma in n_macrostates:
                    thisone = "%s__nstates-%r"%("%s-%d-%s" % ("msmlag-",
                        msm.lag, _model_name(feat, reduce(
                            lambda x, y: dict(x, **y), (
                                {"weights": "empirical" if nm is "ltj" else "koopman"},
                                {"reversible": reversible},
                                {k:v for k,v in setup["kwargs"].items() if k != "lags"})))), nma)
            
                    _fpcca = fpits.with_suffix(".%s-%s-%d__%s" % (
                        clust_method, par_key, setup["par"][1], thisone))

                    if _fpcca.is_file():
                        resultlist.append(pyemma.load(_fpcca))
                    else:
                        result = msm.pcca(nma)
                        result.save(_fpcca)
                        resultlist.append(result)
                
                
                
                
            # TODO PCCA HMM


print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")
"""

MANY PCCA

In [ ]:
"""
for n, (feat, nm, dataset) in enumerate(aswa_tools.iter_models(all_models)):
    dataset["pits"] = dlpits = dict()

    for m, (clust_method, par_key) in enumerate(
        [("regspace", "dmin"), ("kmeans", "k")]
    ):
        dlpits[clust_method] = lpits = list()

        dds = da / nm
        dmsm = dds / "msm" / ("tica_lag_%d" % chosen_tica_lag)
        fits = dmsm / "its-results.pyemma"
        dpits = dds / "pcca" / ("tica_lag_%d" % chosen_tica_lag)
        fpits = dpits / "pits-results.pyemma"
        #fbundle = dpcca / "bundle-results.pyemma"

        if not dpits.is_dir():
            dpits.mkdir(parents=True)

        for i, setup in enumerate(dataset["its"][clust_method]):

            if isinstance(setup["result"], Path):
                print(colorama.Back.CYAN+"Loading file %r"%setup["result"])
                setup["result"] = pyemma.load(setup["result"])
                
            elif not setup["result"]:
                continue
            
            for nma in n_macrostates:
                
                dp = dict()
                dp["par"] = (setup["par"], ("nstates", nma))
                # NOTE THESE KWARGS NOT USED
                #  - pyemma has PCCA object attached to MSM
                #    and just doing stuff to work around
                dp["kwargs"] = {k:v for k,v in setup["kwargs"].items()}
                dp["result"] = resultlist = list()
                
                for msm in setup["result"].models:
                    thisone = "%s__nstates-%r"%("%s-%d-%s" % ("msmlag-",
                        msm.lag, _model_name(feat, reduce(
                            lambda x, y: dict(x, **y), (
                                {"weights": "empirical" if nm is "ltj" else "koopman"},
                                {"reversible": reversible},
                                {k:v for k,v in setup["kwargs"].items() if k != "lags"})))), nma)

                    # stuck one at a time, later use dmgr
                    # to organize on top of its objects
                    _fpcca = fpits.with_suffix(".%s-%s-%d__%s" % (
                        clust_method, par_key, setup["par"][1], thisone))

                    # clear_output(wait=True)
                    print(
                        colorama.Back.CYAN
                        + "Now calculating PCCA+ decomposition of MSM"
                    )
                    print(colorama.Back.CYAN + "Calculated %d models so far" % (m))
                    print((" - on to model %s" % thisone) + colorama.Back.RESET)
                    print(
                        colorama.Fore.LIGHTGREEN_EX
                        + " - number %d / %d (wrong 2nd number)"
                        % (n_kmeans * n + m, (n_kmeans + n_regspaces) * len(all_models))
                    )

                    if _fpcca.is_file():
                        print("Found Results File: {}".format(_fpcca))
                        resultlist.append(pyemma.load(_fpcca))
                    else:
                        result = msm.pcca(nma)
                        result.save(_fpcca)
                        resultlist.append(result)
                        
                        

print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")
"""

### Many HMMs
------------------

 - Save a "failed" file and loook at it if results file not found, change results tag to failed or something

In [1]:
# TODO 
# AS INCREMENTAL FOLLOWING THE PCCA ABOVE!!

howmany = 0
n_kmeans = len(all_models[feat][nm]["cluster_kmeans"])
n_regspaces = len(all_models[feat][nm]["cluster_regspace"])

for feat, nm, dataset in aswa_tools.iter_models(all_models):

    dataset["hits"] = dlits = dict()

    for clust_method, par_key in [("regspace", "dmin"), ("kmeans", "k")]:
        # for clust_method, par_key in [("kmeans", "k")]:
        # for clust_method, par_key in [("regspace", "dmin")]:

        dlits[clust_method] = lits = list()

        dds = da / nm
        dmsm = dds / "hmm" / ("tica_lag_%d" % chosen_tica_lag)
        fits = dmsm / "its-results.pyemma"

        if not dmsm.is_dir():
            dmsm.mkdir(parents=True)

        for n in n_macrostates:
            for i, setup in enumerate(dataset["cluster_%s" % clust_method]):
                # clear_output(wait=True)
                howmany += 1

                thisone = "%s-%s_%d-%s_%s-%s_%s-%s" % (
                    clust_method,
                    "nstates",
                    n,
                    "hmmlags",
                    ",".join([str(d) for d in hmm_lags]),
                    "reversible",
                    str(False),
                    _model_name(feat, setup["kwargs"]),
                )

                print(colorama.Back.CYAN + "Calculated %d models so far" % (howmany))
                print((" - on to model %s" % thisone) + colorama.Back.RESET)
                print(
                    colorama.Fore.LIGHTGREEN_EX
                    + " - number %d / %d (wrong 2nd number)"
                    % (
                        howmany,
                        (n_kmeans + n_regspaces) * n_macrostates[-1] * len(all_models),
                    )
                )

                _fits = fits.with_suffix(".%s" % thisone)

                dits = dict()
                dits["par"] = ((par_key, setup["kwargs"][par_key]), ("nstates", n))

                dits["kwargs"] = dict(
                    lags=hmm_lags,
                    nstates=n,
                    reversible=False,
                    # nits=3,
                    errors="bayes",
                    nsamples=20,
                )

                dits["result"] = False
                print("checking for existing files")

                if not _fits.is_file():

                    print(
                        colorama.Back.MAGENTA
                        + "Calculating and Saving to File: {}".format(_fits)
                        + colorama.Back.RESET
                    )

                    try:
                        #dits["input"] = setup["result"].dtrajs
                        dits["input"] = setup["input"]

                    except AttributeError:
                        print(
                            colorama.Back.RED + "MODEL NOT FOUND" + colorama.Back.RESET
                        )

                    try:
                        # pyemma.msm.timescales_hmsm(
                        dits["result"] = pyemma.msm.timescales_hmsm(
                            dits["input"],
                            **dits["kwargs"]
                            # )
                            # dits["result"].save(_fits)
                        ).save(_fits)

                    except:
                        dits["result"] = False

                else:
                    print(
                        colorama.Back.GREEN
                        + colorama.Fore.RESET
                        + "Found Results File: {}".format(_fits)
                        + colorama.Back.RESET
                    )

                    try:
                        dits["result"] = pyemma.load(_fits)

                    except:
                        dits["result"] = False

                    # dits["result"] = _fits

                if "regspace" in clust_method:
                    lits.insert(0, dits)

                else:
                    lits.append(dits)


print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")

NameError: name 'all_models' is not defined

-----------------

#### $\infty$ DONE
-------

In [ ]:
print(colorama.Back.LIGHTBLACK_EX + colorama.Fore.BLUE + "  DONE LOADING DATA  ")

-----------------

-----------------

------------

------------

------------

-----------

-----------

-----------

-----------

-----------

------------